In [ ]:

# Imports
import torch
import typing
import pydantic
import bittensor as bt

# Stable diffusion
from PIL import Image
from diffusers import StableDiffusionPipeline, StableDiffusionImg2ImgPipeline, StableDiffusionInpaintPipeline
from typing import List, Dict, Union, Tuple, Optional

bt.debug()

# Lets instantiate the stable diffusion model.
model =  StableDiffusionPipeline.from_pretrained( "prompthero/openjourney-v4", custom_pipeline="lpw_stable_diffusion", torch_dtype=torch.float16 )

class TextToImage( bt.Synapse ):
    image: typing.Optional[ bt.Tensor ] = None
    text: str = pydantic.Field( ..., allow_mutation = False)
    height: int = 512
    width: int = 512
    num_images_per_prompt: int = 1 
    num_inference_steps: int = 50
    guidance_scale: float = 7.5 
    negative_prompt: str = None
    seed: int = -1

def text_to_image( synapse: TextToImage ) -> TextToImage:
    return synapse

axon = bt.axon().attach( text_to_image ).start()







In [1]:
# Stable diffusion
import torch
import transformers
from diffusers import StableDiffusionPipeline, StableDiffusionImg2ImgPipeline, StableDiffusionInpaintPipeline

# Lets instantiate the stable diffusion model.
model =  StableDiffusionPipeline.from_pretrained( "prompthero/openjourney-v4", custom_pipeline="lpw_stable_diffusion", torch_dtype=torch.float16 )


/Users/napoli/anaconda3/envs/311/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/napoli/anaconda3/envs/311/lib/python3.11/site-packages/huggingface_hub/file_download.py:649: FutureWarning: 'cached_download' is the legacy way to download files from the HF hub, please consider upgrading to 'hf_hub_download'
  warnings.warn(




Fetching 15 files:  20%|██        | 3/15 [00:01<00:03,  3.24it/s]
























































































































































